In [1]:
#!nvidia-smi
#using a GeForce GTX1080 Ti for reproducibility for all timing experiments

In [2]:
import torch
import torchvision
from torch import nn, optim, autograd
from torch.nn import functional as F
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.autograd import Variable
import numpy as np
from sklearn.utils import shuffle as skshuffle
from math import *
from backpack import backpack, extend
from backpack.extensions import KFAC, DiagHessian, DiagGGNMC
from sklearn.metrics import roc_auc_score
import scipy
from tqdm import tqdm, trange
import pytest
from LB_utils import * 
import time

import matplotlib.pyplot as plt

s = 123
np.random.seed(s)
torch.manual_seed(s)
torch.cuda.manual_seed(s)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cuda_status = torch.cuda.is_available()
print("device: ", device)
print("cuda status: ", cuda_status)

device:  cuda
cuda status:  True


In [4]:

class LPADirNN(nn.Module):
    
    def __init__(self, num_classes=10):
        super(LPADirNN, self).__init__()
        
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, 5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,2),
            torch.nn.Conv2d(32, 64, 5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,2),
            torch.nn.Flatten()     
        )
        self.linear = torch.nn.Linear(4 * 4 * 64, num_classes) 

    def forward(self, x):
        out = self.features(x)
        out = self.linear(out)
        return out

def LPADirNN_last_layer(num_classes=10):
    return(LPADirNN(num_classes=num_classes))

In [5]:
BATCH_SIZE_TRAIN_MNIST = 128
BATCH_SIZE_TEST_MNIST = 128
MAX_ITER_MNIST = 6
LR_TRAIN_MNIST = 10e-6

In [6]:
MNIST_transform = torchvision.transforms.ToTensor()

MNIST_train = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=True,
        download=True,
        transform=MNIST_transform)

mnist_train_loader = torch.utils.data.dataloader.DataLoader(
    MNIST_train,
    batch_size=BATCH_SIZE_TRAIN_MNIST,
    shuffle=True
)


MNIST_test = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform)

mnist_test_loader = torch.utils.data.dataloader.DataLoader(
    MNIST_test,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False,
)

In [7]:
mnist_model = LPADirNN_last_layer().cuda()
loss_function = torch.nn.CrossEntropyLoss()

mnist_train_optimizer = torch.optim.Adam(mnist_model.parameters(), lr=1e-3, weight_decay=5e-4)
MNIST_PATH = "pretrained_weights/MNIST_pretrained_10_classes_last_layer_s{}.pth".format(s)

In [8]:
#Training routine

def train(model, train_loader, optimizer, max_iter, path, verbose=True):
    max_len = len(train_loader)

    for iter in range(max_iter):
        for batch_idx, (x, y) in enumerate(train_loader):
            
            x, y = x.cuda(), y.cuda()
            
            output = model(x)

            accuracy = get_accuracy(output, y)

            loss = loss_function(output, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if verbose and batch_idx % 50 == 0:
                print(
                    "Iteration {}; {}/{} \t".format(iter, batch_idx, max_len) +
                    "Minibatch Loss %.3f  " % (loss) +
                    "Accuracy %.0f" % (accuracy * 100) + "%"
                )

    print("saving model at: {}".format(path))
    torch.save(mnist_model.state_dict(), path)

In [9]:
#train(mnist_model, mnist_train_loader, mnist_train_optimizer, MAX_ITER_MNIST, MNIST_PATH, verbose=True)

In [10]:
#predict in distribution
#MNIST_PATH = "pretrained_weights/MNIST_pretrained_10_classes_last_layer_s{}.pth".format(s)
MNIST_PATH = "pretrained_weights/MNIST_pretrained_10_classes_last_layer.pth"

mnist_model = LPADirNN_last_layer().cuda()
print("loading model from: {}".format(MNIST_PATH))
mnist_model.load_state_dict(torch.load(MNIST_PATH))
mnist_model.eval()

acc = []

max_len = len(mnist_test_loader)
for batch_idx, (x, y) in enumerate(mnist_test_loader):

    x, y = x.cuda(), y.cuda()
    output = mnist_model(x)

    accuracy = get_accuracy(output, y)
    if batch_idx % 10 == 0:
        print(
            "Batch {}/{} \t".format(batch_idx, max_len) + 
            "Accuracy %.0f" % (accuracy * 100) + "%"
        )
    acc.append(accuracy)

avg_acc = np.mean(acc)
print('overall test accuracy on MNIST: {:.02f} %'.format(avg_acc * 100))


loading model from: pretrained_weights/MNIST_pretrained_10_classes_last_layer.pth
Batch 0/79 	Accuracy 100%
Batch 10/79 	Accuracy 97%
Batch 20/79 	Accuracy 98%
Batch 30/79 	Accuracy 100%
Batch 40/79 	Accuracy 100%
Batch 50/79 	Accuracy 100%
Batch 60/79 	Accuracy 100%
Batch 70/79 	Accuracy 98%
overall test accuracy on MNIST: 98.90 %


In [11]:
BATCH_SIZE_TEST_FMNIST = 128
BATCH_SIZE_TEST_KMNIST = 128

In [12]:
FMNIST_test = torchvision.datasets.FashionMNIST(
        '~/data/fmnist', train=False, download=True,
        transform=MNIST_transform)   #torchvision.transforms.ToTensor())

FMNIST_test_loader = torch.utils.data.DataLoader(
    FMNIST_test,
    batch_size=BATCH_SIZE_TEST_FMNIST, shuffle=False)

In [13]:
KMNIST_test = torchvision.datasets.KMNIST(
        '~/data/kmnist', train=False, download=True,
        transform=MNIST_transform)

KMNIST_test_loader = torch.utils.data.DataLoader(
    KMNIST_test,
    batch_size=BATCH_SIZE_TEST_KMNIST, shuffle=False)

In [14]:
"""Load notMNIST"""

import os
import numpy as np
import torch
from PIL import Image
from torch.utils.data.dataset import Dataset
from matplotlib.pyplot import imread
from torch import Tensor

"""
Loads the train/test set. 
Every image in the dataset is 28x28 pixels and the labels are numbered from 0-9
for A-J respectively.
Set root to point to the Train/Test folders.
"""

# Creating a sub class of torch.utils.data.dataset.Dataset
class notMNIST(Dataset):

    # The init method is called when this class will be instantiated
    def __init__(self, root, transform):
        
        #super(notMNIST, self).__init__(root, transform=transform)

        self.transform = transform
        
        Images, Y = [], []
        folders = os.listdir(root)

        for folder in folders:
            folder_path = os.path.join(root, folder)
            for ims in os.listdir(folder_path):
                try:
                    img_path = os.path.join(folder_path, ims)
                    Images.append(np.array(imread(img_path)))
                    Y.append(ord(folder) - 65)  # Folders are A-J so labels will be 0-9
                except:
                    # Some images in the dataset are damaged
                    print("File {}/{} is broken".format(folder, ims))
        data = [(x, y) for x, y in zip(Images, Y)]
        self.data = data
        self.targets = torch.Tensor(Y)

    # The number of items in the dataset
    def __len__(self):
        return len(self.data)

    # The Dataloader is a generator that repeatedly calls the getitem method.
    # getitem is supposed to return (X, Y) for the specified index.
    def __getitem__(self, index):
        img = self.data[index][0]

        if self.transform is not None:
            img = self.transform(img)
            
        # Input for Conv2D should be Channels x Height x Width
        img_tensor = Tensor(img).view(1, 28, 28).float()
        label = self.data[index][1]
        return (img_tensor, label)

In [15]:
#root = os.path.abspath('~/data')
root = os.path.expanduser('~/data')

# Instantiating the notMNIST dataset class we created
notMNIST_test = notMNIST(root=os.path.join(root, 'notMNIST_small'),
                               transform=MNIST_transform)

# Creating a dataloader
not_mnist_test_loader = torch.utils.data.dataloader.DataLoader(
                            dataset=notMNIST_test,
                            batch_size=BATCH_SIZE_TEST_KMNIST,
                            shuffle=False)

File F/Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png is broken
File A/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png is broken


# Prepare Gaussians

In [16]:
t0 = time.time()
M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D = Diag_second_order(model=mnist_model,
                                                                   train_loader=mnist_train_loader,
                                                                   var0 = 1e-3,
                                                                   device=device)
t1 = time.time()
time_gaussian = t1-t0
print("preparing Gaussians took {} seconds which is {} minutes".format(time_gaussian, time_gaussian/60))

n: 1024 inputs to linear layer with m: 10 classes
Batch: 0/469
Batch: 1/469
Batch: 2/469
Batch: 3/469
Batch: 4/469
Batch: 5/469
Batch: 6/469
Batch: 7/469
Batch: 8/469
Batch: 9/469
Batch: 10/469
Batch: 11/469
Batch: 12/469
Batch: 13/469
Batch: 14/469
Batch: 15/469
Batch: 16/469
Batch: 17/469
Batch: 18/469
Batch: 19/469
Batch: 20/469
Batch: 21/469
Batch: 22/469
Batch: 23/469
Batch: 24/469
Batch: 25/469
Batch: 26/469
Batch: 27/469
Batch: 28/469
Batch: 29/469
Batch: 30/469
Batch: 31/469
Batch: 32/469
Batch: 33/469
Batch: 34/469
Batch: 35/469
Batch: 36/469
Batch: 37/469
Batch: 38/469
Batch: 39/469
Batch: 40/469
Batch: 41/469
Batch: 42/469
Batch: 43/469
Batch: 44/469
Batch: 45/469
Batch: 46/469
Batch: 47/469
Batch: 48/469
Batch: 49/469
Batch: 50/469
Batch: 51/469
Batch: 52/469
Batch: 53/469
Batch: 54/469
Batch: 55/469
Batch: 56/469
Batch: 57/469
Batch: 58/469
Batch: 59/469
Batch: 60/469
Batch: 61/469
Batch: 62/469
Batch: 63/469
Batch: 64/469
Batch: 65/469
Batch: 66/469
Batch: 67/469
Batch: 6

In [17]:
M_W_post_K, M_b_post_K, U_post_K, V_post_K, B_post_K = KFLP_second_order(model=mnist_model,
                                                               train_loader=mnist_train_loader,
                                                               var0 = 5e-2,
                                                               device=device)

Batch: 0/469
Batch: 1/469
Batch: 2/469
Batch: 3/469
Batch: 4/469
Batch: 5/469
Batch: 6/469
Batch: 7/469
Batch: 8/469
Batch: 9/469
Batch: 10/469
Batch: 11/469
Batch: 12/469
Batch: 13/469
Batch: 14/469
Batch: 15/469
Batch: 16/469
Batch: 17/469
Batch: 18/469
Batch: 19/469
Batch: 20/469
Batch: 21/469
Batch: 22/469
Batch: 23/469
Batch: 24/469
Batch: 25/469
Batch: 26/469
Batch: 27/469
Batch: 28/469
Batch: 29/469
Batch: 30/469
Batch: 31/469
Batch: 32/469
Batch: 33/469
Batch: 34/469
Batch: 35/469
Batch: 36/469
Batch: 37/469
Batch: 38/469
Batch: 39/469
Batch: 40/469
Batch: 41/469
Batch: 42/469
Batch: 43/469
Batch: 44/469
Batch: 45/469
Batch: 46/469
Batch: 47/469
Batch: 48/469
Batch: 49/469
Batch: 50/469
Batch: 51/469
Batch: 52/469
Batch: 53/469
Batch: 54/469
Batch: 55/469
Batch: 56/469
Batch: 57/469
Batch: 58/469
Batch: 59/469
Batch: 60/469
Batch: 61/469
Batch: 62/469
Batch: 63/469
Batch: 64/469
Batch: 65/469
Batch: 66/469
Batch: 67/469
Batch: 68/469
Batch: 69/469
Batch: 70/469
Batch: 71/469
Ba

# MAP estimate

In [18]:
targets = MNIST_test.targets.numpy()
targets_FMNIST = FMNIST_test.targets.numpy()
targets_notMNIST = notMNIST_test.targets.numpy().astype(int)
targets_KMNIST = KMNIST_test.targets.numpy()

In [19]:
mnist_test_in_MAP = predict_MAP(mnist_model, mnist_test_loader, cuda=True).cpu().numpy()
mnist_test_out_fmnist_MAP = predict_MAP(mnist_model, FMNIST_test_loader, cuda=True).cpu().numpy()
mnist_test_out_notMNIST_MAP = predict_MAP(mnist_model, not_mnist_test_loader, cuda=True).cpu().numpy()
mnist_test_out_KMNIST_MAP = predict_MAP(mnist_model, KMNIST_test_loader, cuda=True).cpu().numpy()

In [20]:
acc_in_MAP, prob_correct_in_MAP, ent_in_MAP, MMC_in_MAP = get_in_dist_values(mnist_test_in_MAP, targets)
acc_out_FMNIST_MAP, prob_correct_out_FMNIST_MAP, ent_out_FMNIST_MAP, MMC_out_FMNIST_MAP, auroc_out_FMNIST_MAP = get_out_dist_values(mnist_test_in_MAP, mnist_test_out_fmnist_MAP, targets_FMNIST)
acc_out_notMNIST_MAP, prob_correct_out_notMNIST_MAP, ent_out_notMNIST_MAP, MMC_out_notMNIST_MAP, auroc_out_notMNIST_MAP = get_out_dist_values(mnist_test_in_MAP, mnist_test_out_notMNIST_MAP, targets_notMNIST)
acc_out_KMNIST_MAP, prob_correct_out_KMNIST_MAP, ent_out_KMNIST_MAP, MMC_out_KMNIST_MAP, auroc_out_KMNIST_MAP = get_out_dist_values(mnist_test_in_MAP, mnist_test_out_KMNIST_MAP, targets_KMNIST)

In [21]:
print_in_dist_values(acc_in_MAP, prob_correct_in_MAP, ent_in_MAP, MMC_in_MAP, 'mnist', 'MAP')
print_out_dist_values(acc_out_FMNIST_MAP, prob_correct_out_FMNIST_MAP, ent_out_FMNIST_MAP, MMC_out_FMNIST_MAP, auroc_out_FMNIST_MAP, 'FMNIST', 'MAP')
print_out_dist_values(acc_out_notMNIST_MAP, prob_correct_out_notMNIST_MAP, ent_out_notMNIST_MAP, MMC_out_notMNIST_MAP, auroc_out_notMNIST_MAP, 'notMNIST', 'MAP')
print_out_dist_values(acc_out_KMNIST_MAP, prob_correct_out_KMNIST_MAP, ent_out_KMNIST_MAP, MMC_out_KMNIST_MAP, auroc_out_KMNIST_MAP, 'KMNIST', 'MAP')

[In, MAP, mnist] Accuracy: 0.989; average entropy: 0.044;     MMC: 0.986; Prob @ correct: 0.100
[Out-MAP, KFAC, FMNIST] Accuracy: 0.063; Average entropy: 1.101;    MMC: 0.604; AUROC: 0.984; Prob @ correct: 0.100
[Out-MAP, KFAC, notMNIST] Accuracy: 0.134; Average entropy: 0.609;    MMC: 0.777; AUROC: 0.914; Prob @ correct: 0.100
[Out-MAP, KFAC, KMNIST] Accuracy: 0.093; Average entropy: 0.725;    MMC: 0.732; AUROC: 0.962; Prob @ correct: 0.100


In [22]:
#MAP estimate
#seeds are 123,124,125,126,127
acc_in = [0.989, 0.989, 0.991, 0.988, 0.989]
mmc_in = [0.988, 0.986, 0.988, 0.988, 0.988]
mmc_out_fmnist = [0.591, 0.604, 0.543, 0.600, 0.581]
mmc_out_notmnist = [0.750, 0.777, 0.775, 0.782, 0.783]
mmc_out_kmnist = [0.717, 0.732, 0.721, 0.725, 0.718]

auroc_out_fmnist = [0.988, 0.984, 0.991, 0.986, 0.988]
auroc_out_notmnist = [0.947, 0.914, 0.937, 0.931, 0.923]
auroc_out_kmnist = [0.969, 0.962, 0.968, 0.968, 0.968]

print("accuracy: {:.03f} with std {:.03f}".format(np.mean(acc_in), np.std(acc_in)))

print("MMC in: {:.03f} with std {:.03f}".format(np.mean(mmc_in), np.std(mmc_in)))
print("MMC out fmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_fmnist), np.std(mmc_out_fmnist)))
print("MMC out notmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_notmnist), np.std(mmc_out_notmnist)))
print("MMC out kmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_kmnist), np.std(mmc_out_kmnist)))

print("AUROC out fmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_fmnist), np.std(auroc_out_fmnist)))
print("AUROC out notmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_notmnist), np.std(auroc_out_notmnist)))
print("AUROC out kmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_kmnist), np.std(auroc_out_kmnist)))

accuracy: 0.989 with std 0.001
MMC in: 0.988 with std 0.001
MMC out fmnist: 0.584 with std 0.022
MMC out notmnist: 0.773 with std 0.012
MMC out kmnist: 0.723 with std 0.005
AUROC out fmnist: 0.987 with std 0.002
AUROC out notmnist: 0.930 with std 0.011
AUROC out kmnist: 0.967 with std 0.003


In [23]:
num_samples = 1000

# Diag Hessian Sampling estimate

In [24]:
mnist_test_in_D = predict_diagonal_sampling(mnist_model, mnist_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True, n_samples=num_samples).cpu().numpy()
mnist_test_out_FMNIST_D = predict_diagonal_sampling(mnist_model, FMNIST_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True, n_samples=num_samples).cpu().numpy()
mnist_test_out_notMNIST_D = predict_diagonal_sampling(mnist_model, not_mnist_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True, n_samples=num_samples).cpu().numpy()
mnist_test_out_KMNIST_D = predict_diagonal_sampling(mnist_model, KMNIST_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True, n_samples=num_samples).cpu().numpy()

time used for sampling with 1000 samples: 6.542214870452881
time used for sampling with 1000 samples: 7.690843105316162
time used for sampling with 1000 samples: 34.59974789619446
time used for sampling with 1000 samples: 13.62425947189331


In [25]:
acc_in_D, prob_correct_in_D, ent_in_D, MMC_in_D = get_in_dist_values(mnist_test_in_D, targets)
acc_out_FMNIST_D, prob_correct_out_FMNIST_D, ent_out_FMNIST_D, MMC_out_FMNIST_D, auroc_out_FMNIST_D = get_out_dist_values(mnist_test_in_D, mnist_test_out_FMNIST_D, targets_FMNIST)
acc_out_notMNIST_D, prob_correct_out_notMNIST_D, ent_out_notMNIST_D, MMC_out_notMNIST_D, auroc_out_notMNIST_D = get_out_dist_values(mnist_test_in_D, mnist_test_out_notMNIST_D, targets_notMNIST)
acc_out_KMNIST_D, prob_correct_out_KMNIST_D, ent_out_KMNIST_D, MMC_out_KMNIST_D, auroc_out_KMNIST_D = get_out_dist_values(mnist_test_in_D, mnist_test_out_KMNIST_D, targets_KMNIST)

In [26]:
print_in_dist_values(acc_in_D, prob_correct_in_D, ent_in_D, MMC_in_D, 'MNIST', 'Diag')
print_out_dist_values(acc_out_FMNIST_D, prob_correct_out_FMNIST_D, ent_out_FMNIST_D, MMC_out_FMNIST_D, auroc_out_FMNIST_D, 'MNIST', test='fmnist', method='Diag')
print_out_dist_values(acc_out_notMNIST_D, prob_correct_out_notMNIST_D, ent_out_notMNIST_D, MMC_out_notMNIST_D, auroc_out_notMNIST_D, 'MNIST', test='notMNIST', method='Diag')
print_out_dist_values(acc_out_KMNIST_D, prob_correct_out_KMNIST_D, ent_out_KMNIST_D, MMC_out_KMNIST_D, auroc_out_KMNIST_D, 'MNIST', test='KMNIST', method='Diag')

[In, Diag, MNIST] Accuracy: 0.989; average entropy: 0.080;     MMC: 0.976; Prob @ correct: 0.100
[Out-fmnist, Diag, MNIST] Accuracy: 0.064; Average entropy: 1.187;    MMC: 0.570; AUROC: 0.980; Prob @ correct: 0.100
[Out-notMNIST, Diag, MNIST] Accuracy: 0.134; Average entropy: 0.731;    MMC: 0.730; AUROC: 0.907; Prob @ correct: 0.100
[Out-KMNIST, Diag, MNIST] Accuracy: 0.093; Average entropy: 0.886;    MMC: 0.666; AUROC: 0.960; Prob @ correct: 0.100


In [27]:
#Diag Sampling (1000)
#seeds are 123,124,125,126,127
time_diag_in = [7.236505508422852, 7.125160217285156, 7.238185882568359, 7.385348320007324, 7.000167608261108]
time_diag_out_fmnist = [7.219660520553589, 8.032358407974243, 7.777910232543945, 7.446442604064941, 7.200972318649292]
time_diag_out_notmnist = [20.68891930580139, 19.327053546905518, 18.424328804016113, 17.584392786026, 17.085043907165527]
time_diag_out_kmnist = [9.627353191375732, 9.133541345596313, 9.307446718215942, 9.410037755966187, 9.100234746932983]

acc_in = [0.989, 0.989, 0.990, 0.988, 0.989]
mmc_in = [0.980, 0.976, 0.979, 0.980, 0.979]
mmc_out_fmnist = [0.558, 0.570, 0.511, 0.566, 0.550]
mmc_out_notmnist = [0.695, 0.730, 0.721, 0.735, 0.731]
mmc_out_kmnist = [0.653, 0.666, 0.654, 0.664, 0.654]

auroc_out_fmnist = [0.985, 0.980, 0.988, 0.983, 0.985]
auroc_out_notmnist = [0.944, 0.907, 0.935, 0.927, 0.921]
auroc_out_kmnist = [0.968, 0.960, 0.967, 0.966, 0.966]

print("Diagonal Sampling time in: {:.03f} with std {:.03f}".format(np.mean(time_diag_in), np.std(time_diag_in)))
print("Diagonal Sampling time out fmnist: {:.03f} with std {:.03f}".format(np.mean(time_diag_out_fmnist), np.std(time_diag_out_fmnist)))
print("Diagonal Sampling time out notmnist: {:.03f} with std {:.03f}".format(np.mean(time_diag_out_notmnist), np.std(time_diag_out_notmnist)))
print("Diagonal Sampling time out kmnist: {:.03f} with std {:.03f}".format(np.mean(time_diag_out_kmnist), np.std(time_diag_out_kmnist)))

print("accuracy: {:.03f} with std {:.03f}".format(np.mean(acc_in), np.std(acc_in)))

print("MMC in: {:.03f} with std {:.03f}".format(np.mean(mmc_in), np.std(mmc_in)))
print("MMC out fmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_fmnist), np.std(mmc_out_fmnist)))
print("MMC out notmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_notmnist), np.std(mmc_out_notmnist)))
print("MMC out kmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_kmnist), np.std(mmc_out_kmnist)))

print("AUROC out fmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_fmnist), np.std(auroc_out_fmnist)))
print("AUROC out notmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_notmnist), np.std(auroc_out_notmnist)))
print("AUROC out kmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_kmnist), np.std(auroc_out_kmnist)))

Diagonal Sampling time in: 7.197 with std 0.129
Diagonal Sampling time out fmnist: 7.535 with std 0.324
Diagonal Sampling time out notmnist: 18.622 with std 1.284
Diagonal Sampling time out kmnist: 9.316 with std 0.193
accuracy: 0.989 with std 0.001
MMC in: 0.979 with std 0.001
MMC out fmnist: 0.551 with std 0.021
MMC out notmnist: 0.722 with std 0.014
MMC out kmnist: 0.658 with std 0.006
AUROC out fmnist: 0.984 with std 0.003
AUROC out notmnist: 0.927 with std 0.013
AUROC out kmnist: 0.965 with std 0.003


# KFAC Laplace Approximation (sampling)

In [28]:
mnist_test_in_KFAC = predict_KFAC_sampling(mnist_model, mnist_test_loader, M_W_post_K, M_b_post_K, U_post_K, V_post_K, B_post_K, verbose=False, cuda=True, timing=True, n_samples=num_samples).cpu().numpy()
mnist_test_out_FMNIST_KFAC = predict_KFAC_sampling(mnist_model, FMNIST_test_loader, M_W_post_K, M_b_post_K, U_post_K, V_post_K, B_post_K, verbose=False, cuda=True, timing=True, n_samples=num_samples).cpu().numpy()
mnist_test_out_notMNIST_KFAC = predict_KFAC_sampling(mnist_model, not_mnist_test_loader, M_W_post_K, M_b_post_K, U_post_K, V_post_K, B_post_K, verbose=False, cuda=True, timing=True, n_samples=num_samples).cpu().numpy()
mnist_test_out_KMNIST_KFAC = predict_KFAC_sampling(mnist_model, KMNIST_test_loader, M_W_post_K, M_b_post_K, U_post_K, V_post_K, B_post_K, verbose=False, cuda=True, timing=True, n_samples=num_samples).cpu().numpy()

RuntimeError: size mismatch, m1: [128 x 1024], m2: [10 x 10] at /opt/conda/conda-bld/pytorch_1587428266983/work/aten/src/THC/generic/THCTensorMathBlas.cu:283

In [ ]:
acc_in_KFAC, prob_correct_in_KFAC, ent_in_KFAC, MMC_in_KFAC = get_in_dist_values(mnist_test_in_KFAC, targets)
acc_out_FMNIST_KFAC, prob_correct_out_FMNIST_KFAC, ent_out_FMNIST_KFAC, MMC_out_FMNIST_KFAC, auroc_out_FMNIST_KFAC = get_out_dist_values(mnist_test_in_KFAC, mnist_test_out_FMNIST_KFAC, targets_FMNIST)
acc_out_notMNIST_KFAC, prob_correct_out_notMNIST_KFAC, ent_out_notMNIST_KFAC, MMC_out_notMNIST_KFAC, auroc_out_notMNIST_KFAC = get_out_dist_values(mnist_test_in_KFAC, mnist_test_out_notMNIST_KFAC, targets_notMNIST)
acc_out_KMNIST_KFAC, prob_correct_out_KMNIST_KFAC, ent_out_KMNIST_KFAC, MMC_out_KMNIST_KFAC, auroc_out_KMNIST_KFAC = get_out_dist_values(mnist_test_in_KFAC, mnist_test_out_KMNIST_KFAC, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_KFAC, prob_correct_in_KFAC, ent_in_KFAC, MMC_in_KFAC, 'MNIST', 'KFAC')
print_out_dist_values(acc_out_FMNIST_KFAC, prob_correct_out_FMNIST_KFAC, ent_out_FMNIST_KFAC, MMC_out_FMNIST_KFAC, auroc_out_FMNIST_KFAC, 'MNIST', test='fmnist', method='KFAC')
print_out_dist_values(acc_out_notMNIST_KFAC, prob_correct_out_notMNIST_KFAC, ent_out_notMNIST_KFAC, MMC_out_notMNIST_KFAC, auroc_out_notMNIST_KFAC, 'MNIST', test='notMNIST', method='KFAC')
print_out_dist_values(acc_out_KMNIST_KFAC, prob_correct_out_KMNIST_KFAC, ent_out_KMNIST_KFAC, MMC_out_KMNIST_KFAC, auroc_out_KMNIST_KFAC, 'MNIST', test='KMNIST', method='KFAC')

# Laplace Bridge

In [ ]:
mnist_test_in_LB = predict_LB(mnist_model, mnist_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True).cpu().numpy()
mnist_test_out_FMNIST_LB = predict_LB(mnist_model, FMNIST_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True).cpu().numpy()
mnist_test_out_notMNIST_LB = predict_LB(mnist_model, not_mnist_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True).cpu().numpy()
mnist_test_out_KMNIST_LB = predict_LB(mnist_model, KMNIST_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True).cpu().numpy()

In [ ]:
mnist_test_in_LBn = mnist_test_in_LB/mnist_test_in_LB.sum(1).reshape(-1,1)
mnist_test_out_FMNIST_LBn = mnist_test_out_FMNIST_LB/mnist_test_out_FMNIST_LB.sum(1).reshape(-1,1)
mnist_test_out_notMNIST_LBn = mnist_test_out_notMNIST_LB/mnist_test_out_notMNIST_LB.sum(1).reshape(-1,1)
mnist_test_out_KMNIST_LBn = mnist_test_out_KMNIST_LB/mnist_test_out_KMNIST_LB.sum(1).reshape(-1,1)

In [ ]:
acc_in_LBn, prob_correct_in_LBn, ent_in_LBn, MMC_in_LBn = get_in_dist_values(mnist_test_in_LBn, targets)
acc_out_FMNIST_LBn, prob_correct_out_FMNIST_LBn, ent_out_FMNIST_LBn, MMC_out_FMNIST_LBn, auroc_out_FMNIST_LBn = get_out_dist_values(mnist_test_in_LBn, mnist_test_out_FMNIST_LBn, targets_FMNIST)
acc_out_notMNIST_LBn, prob_correct_out_notMNIST_LBn, ent_out_notMNIST_LBn, MMC_out_notMNIST_LBn, auroc_out_notMNIST_LBn = get_out_dist_values(mnist_test_in_LBn, mnist_test_out_notMNIST_LBn, targets_notMNIST)
acc_out_KMNIST_LBn, prob_correct_out_KMNIST_LBn, ent_out_KMNIST_LBn, MMC_out_KMNIST_LBn, auroc_out_KMNIST_LBn = get_out_dist_values(mnist_test_in_LBn, mnist_test_out_KMNIST_LBn, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_LBn, prob_correct_in_LBn, ent_in_LBn, MMC_in_LBn, 'MNIST', 'mnist', 'LBn')
print_out_dist_values(acc_out_FMNIST_LBn, prob_correct_out_FMNIST_LBn, ent_out_FMNIST_LBn, MMC_out_FMNIST_LBn, auroc_out_FMNIST_LBn, 'MNIST', test='fmnist', method='LBn')
print_out_dist_values(acc_out_notMNIST_LBn, prob_correct_out_notMNIST_LBn, ent_out_notMNIST_LBn, MMC_out_notMNIST_LBn, auroc_out_notMNIST_LBn, 'MNIST', test='notMNIST', method='LBn')
print_out_dist_values(acc_out_KMNIST_LBn, prob_correct_out_KMNIST_LBn, ent_out_KMNIST_LBn, MMC_out_KMNIST_LBn, auroc_out_KMNIST_LBn, 'MNIST', test='KMNIST', method='LBn')

In [ ]:
#Laplace Bridge
#seeds are 123,124,125,126,127
time_lpb_in = [0.01302, 0.01731, 0.01489, 0.01529, 0.01574]
time_lpb_out_fmnist = [0.01256, 0.01531, 0.01669, 0.01703, 0.01500]
time_lpb_out_notmnist = [0.02332, 0.03115, 0.02666, 0.02756, 0.02864]
time_lpb_out_kmnist = [0.01287, 0.01520, 0.01636, 0.01687, 0.01438]

acc_in = [0.989, 0.989, 0.991, 0.988, 0.989]
mmc_in = [0.988, 0.986, 0.988, 0.988, 0.988]
mmc_out_fmnist = [0.493, 0.523, 0.433, 0.514, 0.488]
mmc_out_notmnist = [0.735, 0.759, 0.756, 0.767, 0.769]
mmc_out_kmnist = [0.699, 0.722, 0.707, 0.713, 0.703]

auroc_out_fmnist = [0.991, 0.988, 0.993, 0.989, 0.990]
auroc_out_notmnist = [0.948, 0.915, 0.938, 0.932, 0.924]
auroc_out_kmnist = [0.970, 0.963, 0.969, 0.968, 0.968]

print("Laplace Bridge time in: {:.03f} with std {:.03f}".format(np.mean(time_lpb_in), np.std(time_lpb_in)))
print("Laplace Bridge time out fmnist: {:.03f} with std {:.03f}".format(np.mean(time_lpb_out_fmnist), np.std(time_lpb_out_fmnist)))
print("Laplace Bridge time out notmnist: {:.03f} with std {:.03f}".format(np.mean(time_lpb_out_notmnist), np.std(time_lpb_out_notmnist)))
print("Laplace Bridge time out kmnist: {:.03f} with std {:.03f}".format(np.mean(time_lpb_out_kmnist), np.std(time_lpb_out_kmnist)))

print("accuracy: {:.03f} with std {:.03f}".format(np.mean(acc_in), np.std(acc_in)))

print("MMC in: {:.03f} with std {:.03f}".format(np.mean(mmc_in), np.std(mmc_in)))
print("MMC out fmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_fmnist), np.std(mmc_out_fmnist)))
print("MMC out notmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_notmnist), np.std(mmc_out_notmnist)))
print("MMC out kmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_kmnist), np.std(mmc_out_kmnist)))

print("AUROC out fmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_fmnist), np.std(auroc_out_fmnist)))
print("AUROC out notmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_notmnist), np.std(auroc_out_notmnist)))
print("AUROC out kmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_kmnist), np.std(auroc_out_kmnist)))

# KFAC Laplace Bridge

In [ ]:
mnist_test_in_LB_KFAC = predict_LB_KFAC(mnist_model, mnist_test_loader, M_W_post_K, M_b_post_K, U_post_K, V_post_K, B_post_K, verbose=False, cuda=True, timing=True).cpu().numpy()
mnist_test_out_FMNIST_LB_KFAC = predict_LB_KFAC(mnist_model, FMNIST_test_loader, M_W_post_K, M_b_post_K, U_post_K, V_post_K, B_post_K, verbose=False, cuda=True, timing=True).cpu().numpy()
mnist_test_out_notMNIST_LB_KFAC = predict_LB_KFAC(mnist_model, not_mnist_test_loader, M_W_post_K, M_b_post_K, U_post_K, V_post_K, B_post_K, verbose=False, cuda=True, timing=True).cpu().numpy()
mnist_test_out_KMNIST_LB_KFAC = predict_LB_KFAC(mnist_model, KMNIST_test_loader, M_W_post_K, M_b_post_K, U_post_K, V_post_K, B_post_K, verbose=False, cuda=True, timing=True).cpu().numpy()

In [ ]:
mnist_test_in_LB_KFACn = mnist_test_in_LB_KFAC/mnist_test_in_LB_KFAC.sum(1).reshape(-1,1)
mnist_test_out_FMNIST_LB_KFACn = mnist_test_out_FMNIST_LB_KFAC/mnist_test_out_FMNIST_LB_KFAC.sum(1).reshape(-1,1)
mnist_test_out_notMNIST_LB_KFACn = mnist_test_out_notMNIST_LB_KFAC/mnist_test_out_notMNIST_LB_KFAC.sum(1).reshape(-1,1)
mnist_test_out_KMNIST_LB_KFACn = mnist_test_out_KMNIST_LB_KFAC/mnist_test_out_KMNIST_LB_KFAC.sum(1).reshape(-1,1)

In [ ]:
acc_in_LB_KFACn, prob_correct_in_LB_KFACn, ent_in_LB_KFACn, MMC_in_LB_KFACn = get_in_dist_values(mnist_test_in_LB_KFACn, targets)
acc_out_FMNIST_LB_KFACn, prob_correct_out_FMNIST_LB_KFACn, ent_out_FMNIST_LB_KFACn, MMC_out_FMNIST_LB_KFACn, auroc_out_FMNIST_LB_KFACn = get_out_dist_values(mnist_test_in_LB_KFACn, mnist_test_out_FMNIST_LB_KFACn, targets_FMNIST)
acc_out_notMNIST_LB_KFACn, prob_correct_out_notMNIST_LB_KFACn, ent_out_notMNIST_LB_KFACn, MMC_out_notMNIST_LB_KFACn, auroc_out_notMNIST_LB_KFACn = get_out_dist_values(mnist_test_in_LB_KFACn, mnist_test_out_notMNIST_LB_KFACn, targets_notMNIST)
acc_out_KMNIST_LB_KFACn, prob_correct_out_KMNIST_LB_KFACn, ent_out_KMNIST_LB_KFACn, MMC_out_KMNIST_LB_KFACn, auroc_out_KMNIST_LB_KFACn = get_out_dist_values(mnist_test_in_LB_KFACn, mnist_test_out_KMNIST_LB_KFACn, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_LB_KFACn, prob_correct_in_LB_KFACn, ent_in_LB_KFACn, MMC_in_LB_KFACn, 'MNIST', 'mnist', 'LB_KFACn')
print_out_dist_values(acc_out_FMNIST_LB_KFACn, prob_correct_out_FMNIST_LB_KFACn, ent_out_FMNIST_LB_KFACn, MMC_out_FMNIST_LB_KFACn, auroc_out_FMNIST_LB_KFACn, 'MNIST', test='fmnist', method='LB_KFACn')
print_out_dist_values(acc_out_notMNIST_LB_KFACn, prob_correct_out_notMNIST_LB_KFACn, ent_out_notMNIST_LB_KFACn, MMC_out_notMNIST_LB_KFACn, auroc_out_notMNIST_LB_KFACn, 'MNIST', test='notMNIST', method='LB_KFACn')
print_out_dist_values(acc_out_KMNIST_LB_KFACn, prob_correct_out_KMNIST_LB_KFACn, ent_out_KMNIST_LB_KFACn, MMC_out_KMNIST_LB_KFACn, auroc_out_KMNIST_LB_KFACn, 'MNIST', test='KMNIST', method='LB_KFACn')

In [ ]:
#Laplace Bridge KFAC
#seeds are 123,124,125,126,127
time_lpb_in = [0.01302, 0.01731, 0.01489, 0.01529, 0.01574]
time_lpb_out_fmnist = [0.01256, 0.01531, 0.01669, 0.01703, 0.01500]
time_lpb_out_notmnist = [0.02332, 0.03115, 0.02666, 0.02756, 0.02864]
time_lpb_out_kmnist = [0.01287, 0.01520, 0.01636, 0.01687, 0.01438]

acc_in = [0.989, 0.989, 0.991, 0.988, 0.989]
mmc_in = [0.988, 0.986, 0.988, 0.988, 0.988]
mmc_out_fmnist = [0.493, 0.523, 0.433, 0.514, 0.488]
mmc_out_notmnist = [0.735, 0.759, 0.756, 0.767, 0.769]
mmc_out_kmnist = [0.699, 0.722, 0.707, 0.713, 0.703]

auroc_out_fmnist = [0.991, 0.988, 0.993, 0.989, 0.990]
auroc_out_notmnist = [0.948, 0.915, 0.938, 0.932, 0.924]
auroc_out_kmnist = [0.970, 0.963, 0.969, 0.968, 0.968]

print("Laplace Bridge time in: {:.03f} with std {:.03f}".format(np.mean(time_lpb_in), np.std(time_lpb_in)))
print("Laplace Bridge time out fmnist: {:.03f} with std {:.03f}".format(np.mean(time_lpb_out_fmnist), np.std(time_lpb_out_fmnist)))
print("Laplace Bridge time out notmnist: {:.03f} with std {:.03f}".format(np.mean(time_lpb_out_notmnist), np.std(time_lpb_out_notmnist)))
print("Laplace Bridge time out kmnist: {:.03f} with std {:.03f}".format(np.mean(time_lpb_out_kmnist), np.std(time_lpb_out_kmnist)))

print("accuracy: {:.03f} with std {:.03f}".format(np.mean(acc_in), np.std(acc_in)))

print("MMC in: {:.03f} with std {:.03f}".format(np.mean(mmc_in), np.std(mmc_in)))
print("MMC out fmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_fmnist), np.std(mmc_out_fmnist)))
print("MMC out notmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_notmnist), np.std(mmc_out_notmnist)))
print("MMC out kmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_kmnist), np.std(mmc_out_kmnist)))

print("AUROC out fmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_fmnist), np.std(auroc_out_fmnist)))
print("AUROC out notmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_notmnist), np.std(auroc_out_notmnist)))
print("AUROC out kmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_kmnist), np.std(auroc_out_kmnist)))

# Conditions

Test the condition derived in Proposition 1 of the paper and evaluated experimentally in Appendix A

In [ ]:
# check if condition holds

def check_condition(alpha_vecs):
    #note that this is vectorized
    alpha_sum = alpha_vecs.sum(1)
    alpha_max = alpha_vecs.max(1)
    alpha_sum_minus = alpha_sum - alpha_max
    right_side = 0.25 * (np.sqrt(9 * alpha_sum_minus**2 + 10 * alpha_sum_minus + 1) - alpha_sum_minus - 1)
    cases = alpha_max > right_side
    percentage = np.sum(cases)/len(cases)
    return(percentage)

In [ ]:
print(np.sum(check_condition(mnist_test_in_LB)))
print(np.sum(check_condition(mnist_test_out_FMNIST_LB)))
print(np.sum(check_condition(mnist_test_out_notMNIST_LB)))
print(np.sum(check_condition(mnist_test_out_KMNIST_LB)))

# Compare to extended MacKay approach

In [ ]:
mnist_test_in_EMK = predict_extended_MacKay(mnist_model, mnist_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True).cpu().numpy()
mnist_test_out_FMNIST_EMK = predict_extended_MacKay(mnist_model, FMNIST_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True).cpu().numpy()
mnist_test_out_notMNIST_EMK = predict_extended_MacKay(mnist_model, not_mnist_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True).cpu().numpy()
mnist_test_out_KMNIST_EMK = predict_extended_MacKay(mnist_model, KMNIST_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True).cpu().numpy()

In [ ]:
acc_in_EMK, prob_correct_in_EMK, ent_in_EMK, MMC_in_EMK = get_in_dist_values(mnist_test_in_EMK, targets)
acc_out_FMNIST_EMK, prob_correct_out_FMNIST_EMK, ent_out_FMNIST_EMK, MMC_out_FMNIST_EMK, auroc_out_FMNIST_EMK = get_out_dist_values(mnist_test_in_EMK, mnist_test_out_FMNIST_EMK, targets_FMNIST)
acc_out_notMNIST_EMK, prob_correct_out_notMNIST_EMK, ent_out_notMNIST_EMK, MMC_out_notMNIST_EMK, auroc_out_notMNIST_EMK = get_out_dist_values(mnist_test_in_EMK, mnist_test_out_notMNIST_EMK, targets_notMNIST)
acc_out_KMNIST_EMK, prob_correct_out_KMNIST_EMK, ent_out_KMNIST_EMK, MMC_out_KMNIST_EMK, auroc_out_KMNIST_EMK = get_out_dist_values(mnist_test_in_EMK, mnist_test_out_KMNIST_EMK, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_EMK, prob_correct_in_EMK, ent_in_EMK, MMC_in_EMK, 'MNIST', 'EMK')
print_out_dist_values(acc_out_FMNIST_EMK, prob_correct_out_FMNIST_EMK, ent_out_FMNIST_EMK, MMC_out_FMNIST_EMK, auroc_out_FMNIST_EMK, 'MNIST', test='fmnist', method='EMK')
print_out_dist_values(acc_out_notMNIST_EMK, prob_correct_out_notMNIST_EMK, ent_out_notMNIST_EMK, MMC_out_notMNIST_EMK, auroc_out_notMNIST_EMK, 'MNIST', test='notMNIST', method='EMK')
print_out_dist_values(acc_out_KMNIST_EMK, prob_correct_out_KMNIST_EMK, ent_out_KMNIST_EMK, MMC_out_KMNIST_EMK, auroc_out_KMNIST_EMK, 'MNIST', test='KMNIST', method='EMK')

In [ ]:
#Extended MacKay
#seeds are 123,124,125,126,127
time_EMK_in = [0.059017181396484375, 0.055175065994262695, 0.04456186294555664, 0.052222251892089844, 0.05633044242858887]
time_EMK_out_fmnist = [0.14669466018676758, 0.06929922103881836, 0.0457310676574707, 0.04770398139953613, 0.04364585876464844]
time_EMK_out_notmnist = [0.06481504440307617, 0.08360505104064941, 0.07899594306945801, 0.0765230655670166, 0.09008359909057617]
time_EMK_out_kmnist = [0.03671598434448242, 0.03630828857421875, 0.04480934143066406, 0.0473024845123291, 0.04279160499572754]

acc_in = [0.989, 0.989, 0.991, 0.988, 0.989]
mmc_in = [0.981, 0.978, 0.981, 0.982, 0.981]
mmc_out_fmnist = [0.564, 0.577, 0.516, 0.573, 0.556]
mmc_out_notmnist = [0.708, 0.741, 0.744, 0.747, 0.744]
mmc_out_kmnist = [0.667, 0.681, 0.669, 0.678, 0.668]

auroc_out_fmnist = [0.986, 0.982, 0.989, 0.984, 0.986]
auroc_out_notmnist = [0.946, 0.909, 0.937, 0.928, 0.922]
auroc_out_kmnist = [0.969, 0.962, 0.968, 0.967, 0.967]

print("Extended MacKay time in: {:.03f} with std {:.03f}".format(np.mean(time_EMK_in), np.std(time_EMK_in)))
print("Extended MacKay time out fmnist: {:.03f} with std {:.03f}".format(np.mean(time_EMK_out_fmnist), np.std(time_EMK_out_fmnist)))
print("Extended MacKay time out notmnist: {:.03f} with std {:.03f}".format(np.mean(time_EMK_out_notmnist), np.std(time_EMK_out_notmnist)))
print("Extended MacKay time out kmnist: {:.03f} with std {:.03f}".format(np.mean(time_EMK_out_kmnist), np.std(time_EMK_out_kmnist)))

print("accuracy: {:.03f} with std {:.03f}".format(np.mean(acc_in), np.std(acc_in)))

print("MMC in: {:.03f} with std {:.03f}".format(np.mean(mmc_in), np.std(mmc_in)))
print("MMC out fmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_fmnist), np.std(mmc_out_fmnist)))
print("MMC out notmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_notmnist), np.std(mmc_out_notmnist)))
print("MMC out kmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_kmnist), np.std(mmc_out_kmnist)))

print("AUROC out fmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_fmnist), np.std(auroc_out_fmnist)))
print("AUROC out notmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_notmnist), np.std(auroc_out_notmnist)))
print("AUROC out kmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_kmnist), np.std(auroc_out_kmnist)))

# Compare to Second-order Delta Posterior Predictive

as detailed in Appendix D

In [ ]:
mnist_test_in_SODPP = predict_SODPP(mnist_model, mnist_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True).cpu().numpy()
mnist_test_out_FMNIST_SODPP = predict_SODPP(mnist_model, FMNIST_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True).cpu().numpy()
mnist_test_out_notMNIST_SODPP = predict_SODPP(mnist_model, not_mnist_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True).cpu().numpy()
mnist_test_out_KMNIST_SODPP = predict_SODPP(mnist_model, KMNIST_test_loader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=True).cpu().numpy()

In [ ]:
acc_in_SODPP, prob_correct_in_SODPP, ent_in_SODPP, MMC_in_SODPP = get_in_dist_values(mnist_test_in_SODPP, targets)
acc_out_FMNIST_SODPP, prob_correct_out_FMNIST_SODPP, ent_out_FMNIST_SODPP, MMC_out_FMNIST_SODPP, auroc_out_FMNIST_SODPP = get_out_dist_values(mnist_test_in_SODPP, mnist_test_out_FMNIST_SODPP, targets_FMNIST)
acc_out_notMNIST_SODPP, prob_correct_out_notMNIST_SODPP, ent_out_notMNIST_SODPP, MMC_out_notMNIST_SODPP, auroc_out_notMNIST_SODPP = get_out_dist_values(mnist_test_in_SODPP, mnist_test_out_notMNIST_SODPP, targets_notMNIST)
acc_out_KMNIST_SODPP, prob_correct_out_KMNIST_SODPP, ent_out_KMNIST_SODPP, MMC_out_KMNIST_SODPP, auroc_out_KMNIST_SODPP = get_out_dist_values(mnist_test_in_SODPP, mnist_test_out_KMNIST_SODPP, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_SODPP, prob_correct_in_SODPP, ent_in_SODPP, MMC_in_SODPP, 'MNIST', 'SODPP')
print_out_dist_values(acc_out_FMNIST_SODPP, prob_correct_out_FMNIST_SODPP, ent_out_FMNIST_SODPP, MMC_out_FMNIST_SODPP, auroc_out_FMNIST_SODPP, 'MNIST', test='fmnist', method='SODPP')
print_out_dist_values(acc_out_notMNIST_SODPP, prob_correct_out_notMNIST_SODPP, ent_out_notMNIST_SODPP, MMC_out_notMNIST_SODPP, auroc_out_notMNIST_SODPP, 'MNIST', test='notMNIST', method='SODPP')
print_out_dist_values(acc_out_KMNIST_SODPP, prob_correct_out_KMNIST_SODPP, ent_out_KMNIST_SODPP, MMC_out_KMNIST_SODPP, auroc_out_KMNIST_SODPP, 'MNIST', test='KMNIST', method='SODPP')

In [ ]:
#SODPP
#seeds are 123,124,125,126,127
time_SODPP_in = [0.022600173950195312, 0.022752046585083008, 0.027388811111450195, 0.0277860164642334, 0.028126955032348633]
time_SODPP_out_fmnist = [0.02283191680908203, 0.022715091705322266, 0.02962970733642578, 0.030440807342529297, 0.028356552124023438]
time_SODPP_out_notmnist = [0.040780067443847656, 0.04070425033569336, 0.04935407638549805, 0.05025911331176758, 0.051819801330566406]
time_SODPP_out_kmnist = [0.021996021270751953, 0.022888660430908203, 0.028764009475708008, 0.030774354934692383, 0.02728438377380371]

acc_in = [0.989, 0.989, 0.990, 0.987, 0.990]
mmc_in = [0.980, 0.977, 0.979, 0.980, 0.979]
mmc_out_fmnist = [0.552, 0.565, 0.506, 0.561, 0.545]
mmc_out_notmnist = [0.681, 0.719, 0.708, 0.724, 0.717]
mmc_out_kmnist = [0.636, 0.648, 0.636, 0.648, 0.636]

auroc_out_fmnist = [0.984, 0.979, 0.987, 0.982, 0.984]
auroc_out_notmnist = [0.945, 0.910, 0.935, 0.928, 0.921]
auroc_out_kmnist = [0.968, 0.960, 0.967, 0.966, 0.966]

print("SODPP time in: {:.03f} with std {:.03f}".format(np.mean(time_SODPP_in), np.std(time_SODPP_in)))
print("SODPP time out fmnist: {:.03f} with std {:.03f}".format(np.mean(time_SODPP_out_fmnist), np.std(time_SODPP_out_fmnist)))
print("SODPP time out notmnist: {:.03f} with std {:.03f}".format(np.mean(time_SODPP_out_notmnist), np.std(time_SODPP_out_notmnist)))
print("SODPP time out kmnist: {:.03f} with std {:.03f}".format(np.mean(time_SODPP_out_kmnist), np.std(time_SODPP_out_kmnist)))

print("accuracy: {:.03f} with std {:.03f}".format(np.mean(acc_in), np.std(acc_in)))

print("MMC in: {:.03f} with std {:.03f}".format(np.mean(mmc_in), np.std(mmc_in)))
print("MMC out fmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_fmnist), np.std(mmc_out_fmnist)))
print("MMC out notmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_notmnist), np.std(mmc_out_notmnist)))
print("MMC out kmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_kmnist), np.std(mmc_out_kmnist)))

print("AUROC out fmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_fmnist), np.std(auroc_out_fmnist)))
print("AUROC out notmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_notmnist), np.std(auroc_out_notmnist)))
print("AUROC out kmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_kmnist), np.std(auroc_out_kmnist)))